In [1]:
from ivrp import *

In [2]:
# load data and random seed
parsed = Parser("IRP_small_instances\IRP_HighT3\S_abs1n5_2_H3.dat")

ivrp = IVRP(parsed.name, parsed.depot, parsed.customers, parsed.vehicles, parsed.nPeriods)

In [3]:
n_cust = len(ivrp.customers)
n_node = n_cust+1
n_veh = len(ivrp.vehicles)
t_period = ivrp.nPeriods+1

In [4]:
A=[]
for i in range(n_cust):
    for j in range(n_cust):
        if i!=j:
            A.append((ivrp.customers[i].id,ivrp.customers[j].id))

In [5]:
c = {(i,j) : cdist([[ivrp.customers[int(i-1)].x, ivrp.customers[int(i-1)].y]], [[ivrp.customers[int(j-1)].x, ivrp.customers[int(j-1)].y]], 'euclidean')[0][0] for (i,j) in A}

In [6]:
for i in range(1,n_node):
    A.append((0,i))
    c[0,i]=cdist([[ivrp.depot.x, ivrp.depot.y]], [[ivrp.customers[int(i-1)].x, ivrp.customers[int(i-1)].y]], 'euclidean')[0][0]
    c[i,0]=c[0,i]


In [7]:
h={}
h[0] = ivrp.depot.h
for i in ivrp.customers:
    h[i.id]=i.h

In [8]:
I=[]
for i in range(1,t_period):
    for j in range(n_node):
        I.append((j,i))

In [9]:
r=ivrp.depot.r

In [10]:
Q=[]
for i in range(1,n_node):
    for k in range(n_veh):
        for t in range(t_period):
            Q.append((i,k,t))

In [11]:
X=[]
for i in range(n_node):
    for j in range(n_node):
        if j!=i:
            for k in range(n_veh):
                for t in range(1,t_period):
                    X.append((i,j,k,t))

In [12]:
d={}
for i in ivrp.customers:
    d[i.id]=i.r

In [13]:
l={}
for i in ivrp.customers:
    l[i.id]=i.l

In [14]:
u={}
for i in ivrp.customers:
    u[i.id]=i.u

In [15]:
from docplex.mp.model import Model
m=Model('ivrp')

# i(i,t) = inventory level of customer i in period t
# q(i,k,t) = qunatity delivered to customer i via vehicle k in period t
# y(i,k,t) = 1 if customer i is visited by vehicle k in period t
# w(i,k,t) = cumulative qty delivered by k up to and incl. cust i in t
# x(i,j,k,t) = 1 if customer j is visited after customer i via k in t
inv = m.continuous_var_dict(I, name='inv')
q = m.continuous_var_dict(Q, lb=0, name='qty')
y = m.binary_var_dict(Q, name='y')
w = m.continuous_var_dict(Q, lb=0, name='w')
x = m.binary_var_dict(X, name='x')

inv[(0,0)]=ivrp.depot.i
for i in ivrp.customers:
    inv[(i.id,0)]=i.i


#(1) Objective function
m.minimize(m.sum(h[i]*inv[(i,t)] + c[(i,j)]*x[(i,j,k,t)] 
                 for i in range(n_node) 
                 for j in range(n_node) if j !=i 
                 for k in range(n_veh) 
                 for t in range(1,t_period)))

#(2) Inv at Depot this period = last period's + production - delivered
m.add_constraints((inv[(0,t)] == inv[(0,t-1)] + r - 
                  (m.sum(q[(i,k,t)] 
                         for i in range(1,n_node)
                         for k in range(n_veh))))
                for t in range(1,t_period))

#(3) Inv at Depot >=0
m.add_constraints(inv[(0,t)]>=0 for t in range(1,t_period))

#(4) Inv at Cust this period = last period's - consumption + delivered
m.add_constraints((inv[(i,t)] == inv[(i,t-1)] - d[i]
                  + m.sum(q[(i,k,t)] 
                         for k in range(n_veh)))
                 for i in range(1,n_node)
                 for t in range(1,t_period))

#(5) Inv at Cust >= lower bound
m.add_constraints((inv[(i,t)]>=l[i])
                 for i in range(1,n_node)
                 for t in range(1,t_period))

#(6) Inv at Cust <= upper bound
m.add_constraints((inv[i,t]<=u[i])
                 for i in range(1,n_node)
                 for t in range(1,t_period))

#(7) Qty delivery exceed space in Cust warehouse (i.e. upper - existing inv)
m.add_constraints(m.sum(q[(i,k,t)] for k in range(n_veh))<= u[i] - inv[(i,t-1)]
                 for i in range(1,n_node)
                 for t in range(1,t_period))

#(8) If x is 1, qty delivered<capacity, if x is 0, qty delivered=0
m.add_constraints((m.sum(q[(i,k,t)] for k in range(n_veh)) <= 
                  u[i]*m.sum((x[(i,j,k,t)] 
                        for j in range(1,n_node) if j!=i
                        for k in range(n_veh))))
                  for i in range(1,n_node)
                  for t in range(1,t_period))

#(9) Qty to be delivered by each vehicle within vehicle's capacity
m.add_constraints(m.sum(q[(i,k,t)] for i in range(1,n_node)) <= 
                  ivrp.vehicles[k].Q 
                  for k in range(n_veh) 
                  for t in range(1,t_period))
                  
#(10) qty delivered to each cust is below capacity if visited, and 0 if not visited
m.add_constraints(q[(i,k,t)]<=(y[(i,k,t)]*u[i]) 
                  for i in range(1,n_node)
                  for k in range(n_veh)
                  for t in range(1,t_period))

#(11) there must be a node before and after each visited cust 
m.add_constraints((m.sum(x[(i,j,k,t)] for j in range(n_node) if i!=j) ==
                   m.sum(x[(j,i,k,t)] for j in range(n_node) if j!=i)) 
                  for i in range(1,n_node)
                  for k in range(n_veh)
                  for t in range(1,t_period))
m.add_constraints((m.sum(x[(i,j,k,t)] for j in range(n_node) if i!=j) ==
                   m.sum(y[(i,k,t)])) 
                  for i in range(1,n_node)
                  for k in range(n_veh)
                  for t in range(1,t_period))
m.add_constraints((m.sum(x[(j,i,k,t)] for j in range(n_node) if j!=i) ==
                   m.sum(y[(i,k,t)])) 
                  for i in range(1,n_node)
                  for k in range(n_veh)
                  for t in range(1,t_period))
#m.add_constraints((m.sum(x[(i,j,k,t)] for j in range(n_node) if i!=j) ==
#                   m.sum(x[(j,i,k,t)] for j in range(n_node) if j!=i) ==
#                   m.sum(y[(i,k,t)])) 
#                  for i in range(1,n_node)
#                  for k in range(n_veh)
#                  for t in range(1,t_period))

#(12) at most one route from depot
m.add_constraints(m.sum(x[(0,j,k,t)] for j in range(1,n_node)) <= 1 
                  for k in range(n_veh) 
                  for t in range(1,t_period))

#(13) at most one veh visited each cust 
m.add_constraints(m.sum(y[(i,k,t)] for k in range(n_veh)) <= 1
                  for i in range(1,n_node) 
                  for t in range(1,t_period))

#(14) routing delivery qty flow 
m.add_constraints(w[(i,k,t)] - w[(j,k,t)] + (ivrp.vehicles[k].Q)*x[(i,j,k,t)] 
                  <= ivrp.vehicles[k].Q - q[(j,k,t)] 
                  for i in range(1,n_node) 
                  for j in range(1,n_node) if j!=i 
                  for k in range(n_veh) 
                  for t in range(1,t_period))

#(15) variable logic
m.add_constraints(q[(i,k,t)] <= w[(i,k,t)] 
                  for i in range(1,n_node) 
                  for k in range(n_veh) 
                  for t in range(1,t_period))
m.add_constraints(w[(i,k,t)] <= ivrp.vehicles[k].Q 
                  for i in range(1,n_node) 
                  for k in range(n_veh) 
                  for t in range(1,t_period))
m.add_constraints(q[(i,k,t)] <= ivrp.vehicles[k].Q 
                  for i in range(1,n_node) 
                  for k in range(n_veh) 
                  for t in range(1,t_period))
#m.add_constraints(q[(i,k,t)] <= w[(i,k,t)] <= ivrp.vehicles[k].Q 
#                  for i in range(1,n_node) 
#                  for k in range(n_veh) 
#                  for t in range(1,t_period))

#(16) & (17)
# covered in var definition
solution = m.solve(log_output = False)
print(solution)

solution for: ivrp
objective: 8815.52
inv_0_1=638.000
inv_1_1=130.000
inv_2_1=35.000
inv_4_1=24.000
inv_0_2=687.000
inv_1_2=65.000
inv_2_2=35.000
inv_4_2=29.000
inv_5_2=11.000
inv_0_3=822.000
inv_4_3=5.000
qty_1_1_1=65.000
qty_2_0_2=35.000
qty_3_0_2=58.000
qty_3_0_3=58.000
qty_4_0_2=29.000
qty_5_0_2=22.000
y_1_0_2=1
y_1_0_3=1
y_1_1_1=1
y_2_0_2=1
y_3_0_2=1
y_3_0_3=1
y_3_1_1=1
y_4_0_2=1
y_5_0_2=1
w_1_0_2=144.000
w_1_0_3=144.000
w_1_1_1=65.000
w_2_0_2=122.000
w_3_0_1=144.000
w_3_0_2=58.000
w_3_0_3=58.000
w_3_1_1=65.000
w_4_0_2=87.000
w_5_0_2=144.000
x_0_1_1_1=1
x_0_3_0_2=1
x_0_3_0_3=1
x_1_0_0_2=1
x_1_0_0_3=1
x_1_3_1_1=1
x_2_5_0_2=1
x_3_0_1_1=1
x_3_1_0_3=1
x_3_4_0_2=1
x_4_2_0_2=1
x_5_1_0_2=1

